In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from aind_vr_foraging_analysis.utils.parsing import data_access
from aind_vr_foraging_analysis.utils.parsing import parse

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

import matplotlib.patches as mpatches
# Plotting libraries
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, FuncFormatter

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1="#FF6F01"
color2='#1b9e77'
color3='#7570b3'
color4='yellow'
odor_list_color = [color1, color2, color3, color4]
import matplotlib.lines as mlines

pdf_path = r'Z:\scratch\vr-foraging\sessions'
results_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\experiments\batch 5 - learning\results'

color_dict_label = {'InterSite': '#808080',
    'InterPatch': '#b3b3b3', 
    'PatchZ': '#d95f02', 'PatchZB': '#d95f02', 
    'PatchB': '#d95f02','PatchA': '#7570b3', 
    'PatchC': '#1b9e77',
    'Alpha-pinene': '#1b9e77', 
    'Methyl Butyrate': '#7570b3', 
    'Amyl Acetate': '#d95f02', 
    'Fenchone': '#d95f02', 
    'S': color1,
    'D': color2,
    'N': color3,   
    }

label_dict = {**{
"InterSite": '#808080',
"InterPatch": '#b3b3b3'}, 
            **color_dict_label}
import os
import re
sns.set_context("talk")

In [ ]:
trainer_dict = {
                '789909': '3_Huy',
                '789910': '3_Huy',
                '789911': '3_Huy',
                '754579': '3_Huy',
                '789918': '3_Huy',
                '789919': '3_Huy',
                '789907': '2_Olivia',
                '789903': '2_Olivia',
                '789925': '2_Olivia',
                '789924': '2_Olivia',
                '789926': '2_Olivia',
                '789908': '2_Olivia',
                '754574': '1_Katrina',
                '789914': '1_Katrina', 
                '789915': '1_Katrina', 
                '789923': '1_Katrina', 
                '789917' : '1_Katrina', 
                '789913' : '1_Katrina', 
}      
mouse_list = trainer_dict.keys()    

In [ ]:
date_string = "2025-5-16"

sum_df = pd.DataFrame()
for mouse in mouse_list:
    with PdfPages(os.path.join(results_path, f"{mouse}_reversal_{date_string}.pdf")) as pdf:
        session_paths = data_access.find_sessions_relative_to_date(
            mouse=mouse,
            date_string=date_string,
            when='on'
        )
        session_n = 0
        for session_path in session_paths:
            print(mouse, session_path)
            try:
                all_epochs, stream_data, data = data_access.load_session(
                    session_path
                )
            
                data['harp_environment_sensor'].streams.SensorData.load_from_file()
                df = data['harp_environment_sensor'].streams.SensorData.data.reset_index()
                # Align to beninning of session
                df['Time'] = df['Time'] - df['Time'][0]
                rig = data['config'].streams.rig_input.data            
                df['rig'] = rig['rig_name']
                df['mouse'] = mouse
                df['shift'] = trainer_dict[mouse]
                
                # common_time = np.arange(df['Time'].min(), df['Time'].max(), 0.01)  # e.g., dt = 0.01 seconds
                # interpolated = df.set_index('Time')[['Temperature', 'Pressure', 'Humidity']].reindex(common_time).interpolate(method='linear')
                # interpolated.index.name = 'Time'
                # interpolated['rig'] = rig['rig_name']

                sum_df = pd.concat([sum_df, df])
            
            except:
                print(f"Error loading {session_path.name}")
                continue

In [ ]:
# Room temperature recordings
session_path = r'Z:\scratch\vr-foraging\temperature_validation\000000_2025-05-15T234720Z'
data = parse.load_session_data(session_path)
data['harp_environment_sensor'].streams.SensorData.load_from_file()
df = data['harp_environment_sensor'].streams.SensorData.data.reset_index()
df['rig'] = 'room'
df['shift'] = 'morning'

sum_df = pd.concat([sum_df, df])

session_path = r'Z:\scratch\vr-foraging\temperature_validation\000000_2025-05-16T180326Z'
data = parse.load_session_data(session_path)
data['harp_environment_sensor'].streams.SensorData.load_from_file()
df = data['harp_environment_sensor'].streams.SensorData.data.reset_index()
df['rig'] = 'room'
df['shift'] = 'noon'

sum_df = pd.concat([sum_df, df])

session_path = r'Z:\scratch\vr-foraging\temperature_validation\000000_2025-05-16T221737Z'
data = parse.load_session_data(session_path)
data['harp_environment_sensor'].streams.SensorData.load_from_file()
df = data['harp_environment_sensor'].streams.SensorData.data.reset_index()
df['rig'] = 'room'
df['shift'] = 'afternoon'

sum_df = pd.concat([sum_df, df])

In [ ]:
sum_df.sort_values(by=['shift', 'rig'], inplace=True)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(14, 5), sharey=True)
for i, (ax, trainer, control) in enumerate(zip(axes.flatten(), sum_df['shift'].unique(), ['morning', 'noon', 'afternoon'])):
    print(trainer, control)
    sns.lineplot(data=sum_df.loc[sum_df['shift']==trainer], x='Time', y='Temperature', hue='rig',  palette='Set1', errorbar=None,  legend=(i == 0), ax=ax)
    value = sum_df.loc[sum_df['shift']==control].Temperature.mean()
    print(value)
    ax.scatter(0, value, color='black')
    ax.set_title(trainer)
    ax.set_ylim(22, 32)
sns.despine()
# Extract legend handles and labels from the last axis
handles, labels = axes[0].get_legend_handles_labels()

# Remove the axis-level legend completely
axes[0].get_legend().remove()

# Create a single shared legend
plt.legend(
    handles=handles,
    labels=labels,
    title='Rig',
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    ncol=2
)
plt.suptitle(date_string)
plt.tight_layout()

### Compare turning off everything overnight to not doing it
5B was turn off overnight on the 5-16-2025.There were many issues during start up so there was roughly 20 min between first turn on to actually acquisition. 

In [ ]:
mouse_list = ['789915']
date_string = "2025-5-12"
sum_df = pd.DataFrame()
for mouse in mouse_list:
    with PdfPages(os.path.join(results_path, f"{mouse}_reversal_{date_string}.pdf")) as pdf:
        session_paths = data_access.find_sessions_relative_to_date(
            mouse=mouse,
            date_string=date_string,
            when='on_or_after'
        )
        session_n = 0
        for session_path in session_paths:
            print(mouse, session_path)
            try:
                all_epochs, stream_data, data = data_access.load_session(
                    session_path
                )
            
                data['harp_environment_sensor'].streams.SensorData.load_from_file()
                df = data['harp_environment_sensor'].streams.SensorData.data.reset_index()
                # Align to beninning of session
                df['Time'] = df['Time'] - df['Time'][0]
                rig = data['config'].streams.rig_input.data   
                if session_path.name.split('_')[1] == '2025-05-16T165040Z' and mouse == '789915':
                    df['rig'] = rig['rig_name']+'_OFF'
                else:
                    df['rig'] = rig['rig_name']+'_ON'
                df['date'] = session_path.name.split('_')[1]           
                         
                # common_time = np.arange(df['Time'].min(), df['Time'].max(), 0.01)  # e.g., dt = 0.01 seconds
                # interpolated = df.set_index('Time')[['Temperature', 'Pressure', 'Humidity']].reindex(common_time).interpolate(method='linear')
                # interpolated.index.name = 'Time'
                # interpolated['rig'] = rig['rig_name']

                sum_df = pd.concat([sum_df, df])
            
            except:
                print(f"Error loading {session_path.name}")
                continue

In [ ]:
sns.lineplot(data=sum_df, x='Time', y='Temperature', hue='date',  style='rig', palette='Set1', errorbar=None)
plt.legend(title='Date', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)
plt.title(sum_df.rig.unique()[0][:2])
sns.despine()